In [1]:
import joblib
import numpy as np
import pandas as pd
import gc
import time
import os
import sys
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization
from skopt import BayesSearchCV 
# from optbinning import OptimalBinning
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from yellowbrick.cluster import KElbowVisualizer
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
import random
import itertools
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [39]:
! pip install optbinning
from optbinning import OptimalBinning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.2/211.2 kB 5.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 40.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.2/296.2 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 82.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.5 MB/s eta 0:00:00


In [2]:
pd.set_option('display.max_rows', 500)

In [30]:
# read data
# train = pd.read_csv('/kaggle/input/dacon001/open/train.csv')
# test = pd.read_csv('/kaggle/input/dacon001/open/test.csv')
train = pd.read_csv('/kaggle/input/groupdata2/train_w_group.csv')
test = pd.read_csv('/kaggle/input/groupdata2/test_w_group.csv')
sample_submission = pd.read_csv('/kaggle/input/dacon001/open/sample_submission.csv')
original_columns = test.columns.tolist()

In [31]:
# test데이터셋에 있는 변수만 선택 
train = train[test.columns.to_list()+['Y_LABEL','SAMPLE_TRANSFER_DAY']].copy()

In [32]:
cat_features  = ['YEAR','COMPONENT_ARBITRARY']
num_features13 = ['AG', 'CO', 'CR', 'CU', 'FE', 'H2O', 'MN', 'MO', 'NI',  'TI', 'V', 'V40', 'ZN']
num_features3 = ['ANONYMOUS_1', 'ANONYMOUS_2','PQINDEX'] 
all_num_features = num_features13 + num_features3
타겟과음의상관관계 = ['ZN','H2O'] # 타겟과 마이너스 상관관계

In [34]:
print(len(train.columns))
vi

1237


In [53]:
iv_list = []
for variable in [i for i in train.columns if i not in ['Y_LABEL','ID','COMPONENT_ARBITRARY']]:
    
    optb = OptimalBinning(name=variable, dtype="numerical", solver="cp")

    x = train[variable].values
    y = train['Y_LABEL']

    optb.fit(x, y)

    binning_table = optb.binning_table
    a1 = binning_table.build()

    iv = a1['IV'].sum()
    iv_list.append([variable,iv])

In [59]:
iv_df = pd.DataFrame(iv_list,columns=['feature','iv'])
iv_df = iv_df.sort_values('iv',ascending=False)
iv_df.head(50)

,feature,iv
413,ANONYMOUS_1_ANONYMOUS_2_div,0.366576
533,ANONYMOUS_1_ANONYMOUS_2_logdiff,0.359434
173,ANONYMOUS_1_ANONYMOUS_2_minus,0.357444
297,ANONYMOUS_1_NI_plus,0.349813
18,ZN_ANONYMOUS_1_mul,0.346922
245,ANONYMOUS_2_NI_minus,0.332064
605,ANONYMOUS_2_NI_logdiff,0.331380
166,ANONYMOUS_1_TI_minus,0.324546
526,ANONYMOUS_1_TI_logdiff,0.322275
417,ANONYMOUS_1_NI_div,0.322122
